In [124]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from fuzzywuzzy import fuzz, process


C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [131]:
today = f"{datetime.now():%Y%m%d}"
open_file = open(f'{today}_datadict.pkl', "rb")
datadict = pickle.load(open_file)
open_file.close()

In [152]:
open_file = open(f'scores.pkl', "rb")
scores = pickle.load(open_file)
open_file.close()

In [123]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [132]:
res = []
for dic in datadict:
    e_dict = {}
    for name, tup in dic.items():
        name = name.replace(' ','_').lower()
        if len(tup) == 2:
            e_dict[f'{name}_diff'] = float(tup[0]) - float(tup[1])
            e_dict[f'{name}_sum'] = float(tup[0]) + float(tup[1])
        else:
            e_dict[name] = tup
    if 'bet' in e_dict:
        e_dict['odds_sum'] = e_dict['bet']['odds_checker_sum']
        del e_dict['bet']
    e_dict['median_odds'] = e_dict.pop('odds')
    if len(e_dict['median_odds']) > 0:
        e_dict['std_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                      [np.std(sorted(x)[1:-1]) for x in e_dict['full_odds']]))
    res.append(e_dict)

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [149]:
df = pd.DataFrame(res)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['median_odds'] != '') & (df['median_odds'] != {})]
df = df.reset_index(drop = True)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
std_o = []
for i in df[['std_odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    std_o.append(oo)
r = []
for i in df[['median_odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    r.append(oo)    
df['med_o'] = r
df['std_o'] = std_o
df['odds_predict'] = df['med_o'].apply(lambda x: np.argmin(x))
ordo_df = pd.DataFrame(df['med_o'].tolist(), columns=['med_0', 'med_1', 'med_2'])
std_o = pd.DataFrame(df['std_o'].tolist(), columns=['std_0', 'std_1', 'std_2'])
df = pd.concat([df, std_o, ordo_df], axis=1)
df = df.drop(['match_name', 'half_time','odds_url','o_names','med_o', 'std_o', 'std_odds',
             'median_odds','url','full_odds'], axis = 1)

In [154]:
df['target'] = scores

In [156]:
df.to_csv('f_eng_df.csv')